In [2]:
import pandas as pd

df = pd.read_csv('Dataset\IMDB-Dataset.csv')

### Split Train/Val and Vectorize

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['sentiment'])
X = df['review_processed'].values.tolist()

x_train_tmp, x_val_tmp, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=0
)

tfidt = TfidfVectorizer(max_features=20000)
tfidt.fit(x_train_tmp, y_train)

x_train = tfidt.transform(x_train_tmp)
x_val = tfidt.transform(x_val_tmp)

### Training

In [9]:
from sklearn.metrics import f1_score

#### Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(
    criterion='entropy',
    random_state = 42
)

dt_clf.fit(x_train, y_train)

y_pred = dt_clf.predict(x_val)

In [18]:
print(f'F1 score: {f1_score(y_val, y_pred)}')

F1 score: 0.7203970223325062


#### Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    random_state = 42
)

rf_clf.fit(x_train, y_train)

y_pred = rf_clf.predict(x_val)

In [20]:
print(f'F1 score: {f1_score(y_val, y_pred)}')

F1 score: 0.8496784565916399


#### XGBoost

In [21]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(n_estimators = 500, # số lượng cây
                        learning_rate = 0.05, 
                        use_label_encoder = False,
                        eval_metric = "logloss",
                        early_stopping_rounds = 5, # ngăn overfitting
                        n_jobs = -1) # dùng toàn bộ cpu cores

xgb_clf.fit(x_train, y_train,                    
            eval_set = [(x_val,y_val)],
            verbose = 1)

d:\anaconda3\envs\nlp_env\lib\site-packages\xgboost\core.py:158: UserWarning: [11:25:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-logloss:0.68169
[1]	validation_0-logloss:0.67137
[2]	validation_0-logloss:0.66202
[3]	validation_0-logloss:0.65341
[4]	validation_0-logloss:0.64548
[5]	validation_0-logloss:0.63816
[6]	validation_0-logloss:0.63143
[7]	validation_0-logloss:0.62509
[8]	validation_0-logloss:0.61920
[9]	validation_0-logloss:0.61365
[10]	validation_0-logloss:0.60839
[11]	validation_0-logloss:0.60341
[12]	validation_0-logloss:0.59864
[13]	validation_0-logloss:0.59417
[14]	validation_0-logloss:0.58995
[15]	validation_0-logloss:0.58587
[16]	validation_0-logloss:0.58209
[17]	validation_0-logloss:0.57859
[18]	validation_0-logloss:0.57526
[19]	validation_0-logloss:0.57191
[20]	validation_0-logloss:0.56865
[21]	validation_0-logloss:0.56567
[22]	validation_0-logloss:0.56253
[23]	validation_0-logloss:0.55983
[24]	validation_0-logloss:0.55711
[25]	validation_0-logloss:0.55439
[26]	validation_0-logloss:0.55196
[27]	validation_0-logloss:0.54929
[28]	validation_0-logloss:0.54700
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [23]:
print(f'F1 score: {f1_score(xgb_clf.predict(x_val), y_pred)}')

F1 score: 0.9176470588235294
